# Principal Component Analysis

The idea behind PCA is to transform our dataset into something more useful for building models. What we want to do is to build new dimensions (predictors) out of the dimensions we are given in such a way that:

(1) each dimension we draw captures as much of the remaining variance among our predictors as possible; and <br/>
(2) each dimension we draw is orthogonal to the ones we've already drawn.

## Motivation

Think back to multiple linear regression for a moment.

The fundamental idea is that I can get a better prediction for my dependent variable by considering a *linear combination of my predictors* than I can get by considering any one predictor by itself.

$\rightarrow$ **PCA insight**: If the combinations of predictors work better than the predictors themselves, then let's just treat the combinations as our primary dimensions!

But one problem with having lots of predictors is that it raises the chance that some will be nearly *collinear*.

$\rightarrow$ **PCA insight**: Since we're reconstructing our dimensions anyway, let's make sure that the dimensions we construct are mutually orthogonal! <br/>
$\rightarrow$ **PCA insight**: Moreover, since we'll be capturing much of the variance among our predictors in the first few dimensions we construct, we'll be able in effect to *reduce  the dimensionality* of our problem. Thus PCA is a fundamental tool in *dimensionality reduction*.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

In [ ]:
cars = pd.read_csv('../../randomforest/Random-Forest_tree_ensembles/cars.csv')
cars.info()

In [ ]:
cars.head()

In [ ]:
cars[' cubicinches'].replace(' ', np.nan, inplace=True)
cars[' cubicinches'] = cars[' cubicinches'].map(float)
cars[' cubicinches'].fillna(cars[' cubicinches'].mean(skipna=True), inplace=True)

In [ ]:
cars[' weightlbs'].replace(' ', np.nan, inplace=True)
cars[' weightlbs'] = cars[' weightlbs'].map(float)
cars[' weightlbs'].fillna(cars[' weightlbs'].mean(), inplace=True)

In [ ]:
cars[' cylinders'] = cars[' cylinders'].map(float)
cars[' hp'] = cars[' hp'].map(float)
cars[' time-to-60'] = cars[' time-to-60'].map(float)
cars[' year'] = cars[' year'].map(float)

In [ ]:
# Let's define our predictors and target
X = cars.drop([' brand','mpg'], axis=1)
y = cars['mpg']

In [ ]:
# Splitting

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
# Scaling
ss = StandardScaler().fit(X_train)


In [ ]:
# Scale-transforming

X_tr_sc = ss.transform(X_train)
X_ts_sc = ss.transform(X_test)

In [ ]:
# Let's construct a linear regression
lr = LinearRegression().fit(X_tr_sc,y_train)
# Score on train
lr.score(X_tr_sc, y_train)

In [ ]:
# Score on test
lr.score(X_ts_sc, y_test)

In [ ]:
# Get the coefficients of the best-fit hyperplane
lr.coef_


Thus, our best-fit hyperplane is given by:

$-1.404\times cyl + 0.669\times in^3 - 0.482\times hp - 4.651\times lbs. -  0.177\times time_{60} + 2.425\times yr$

## Eigenvalues and Eigenvectors

The key idea is to diagonalize (i.e. find the eigendecomposition of) the covariance matrix. The decomposition will produce a set of orthogonal vectors that explain as much of the remaining variance as possible.

Let's say a word about eigenvalues and eigenvectors. It turns out that eigenvalues and -vectors have a dizzying number of applications. But the basic idea is that, if we can split a bunch of vectors (i.e. a matrix) into a set of mutually orthogonal vectors, then we can isolate the force of the bunch into discrete bits, each of which by itself acts like a simple linear transformation.

That's why the definition of an eigenvector is as it is: $\vec{x}$ is an eigenvector of the matrix $A$ if $A\vec{x} = \lambda\vec{x}$, for some scalar $\lambda$. That is, the vector is oriented in just such a direction that multiplying the matrix by it serves only to lengthen or shorten it.

Let's do a simple example.

Suppose we have the matrix
$A =
\begin{bmatrix}
1 & 0.6 \\
0.6 & 1 \\
\end{bmatrix}
$

Let's calculate the eigendecomposition of this matrix.


## PCA by Hand
[Here's](https://www.youtube.com/watch?v=_UVHneBUBW0) a video introduction to PCA.

In [ ]:

A = [[1,0.6],[0.6,1]]

values, vectors = np.linalg.eig(A) #eigen vector is the output vector


In [ ]:
vectors.dot(np.diag(values)).dot(vectors.T )   #should get back A

N.B.: What follows is indebted to http://sebastianraschka.com/Articles/2015_pca_in_3_steps.html#pca-vs-lda

In [ ]:
# We'll start by producing the covariance matrix for the columns of X.
cov_mat = np.cov(X_tr_sc.T) #shows the relationship among the cols

In [ ]:
cov_mat

In [ ]:
# np.linalg.eig(X) returns a double of NumPy arrays, the first containing
# the eigenvalues of X and the second containing the eigenvectors of X.



In [ ]:
# Let's assign the results of eig(cov_mat) to a double of variables.

eigvals, eigvecs = np.linalg.eig(cov_mat)

In [ ]:
# We'll now pair up each eigenvalue with its corresponding eigenvector.

eigpairs = [(eigvals[i], eigvecs[:,i]) for i in range(len(eigvals))]   

In [ ]:
# Let's look at the first element of eigpairs.

eigpairs[0]

In [ ]:
# The second element of each element in eigpairs is
# an eigenvector of the covariance matrix.

eigpairs[0][1]

In [ ]:
# We want to isolate the eigenvectors and create a matrix
# with a column for each. We'll use just three of these,
# corresponding to taking the first three principal components. 
# they are ordered so the first three are the most significant

pcabh = np.hstack((eigpairs[0][1].reshape(6,1),
                  eigpairs[1][1].reshape(6,1),
                  eigpairs[2][1].reshape(6,1)))
pcabh

In [ ]:
# Now we simply want the dot-product of
# X (scaled) with this matrix of the eigenvectors
# of the covariance matrix of X.

X_tr_sc.dot(pcabh)

In [ ]:
# Naturally, sklearn has a shortcut for this!

pca = PCA(n_components=3)
X_train_new = pca.fit_transform(X_tr_sc)
X_train_new

In [ ]:
# Let's check out the explained variance
pca.explained_variance_


In [ ]:
# The ratio is often more informative
pca.explained_variance_ratio_


In [ ]:
# We can also check out the Principal Components themselves

pca.components_

In [ ]:
# Recall the columns of X

X.columns

The results of our PCA are as follows:

**PC1** = 0.454 * cylinders + 0.470 * cubicinches + 0.462 * hp + 0.440 * weightlbs - 0.357 * time-to-60 - 0.196 * year

**PC2** = -0.143 * cylinders - 0.110 * cubicinches - 0.023 * hp - 0.217 * weightlbs - 0.102 * time-to-60 - 0.954 * year

**PC3** = 0.204 * cylinders + 0.153 * cubicinches - 0.129 * hp + 0.361 * weightlbs + 0.860 * time-to-60 - 0.220 * year

It turns out that these loadings are encoded in the eigenvectors of $X^TX$. Notice that:

- the absolute values of the components of PC1 are the first components of the eigenvectors below, <br/>
- the absolute values of the components of PC2 are the second components of the eigenvectors below, <br/>
- etc. <br/>

We'll have more to say about this when we examine the singular value decomposition of matrices in Mod 4.

In [ ]:
np.linalg.eig(X_tr_sc.T.dot(X_tr_sc))

## Normality

In [ ]:
# These principal components should be normalized.
# If they are, then the sum of the squares of the
# loadings should be 1. Let's check!

mag0 = 0 
for i in range(6):
    mag0+=pca.components_[0][i]**2
mag0

In [ ]:
mag1 = 0 
for i in range(6):
    mag1+=pca.components_[1][i]**2
mag1

In [ ]:
mag2 = 0 
for i in range(6):
    mag2+=pca.components_[2][i]**2
mag2

## Orthogonality

In [ ]:
# These principal components should also be
# mutually orthogonal. If they are, then the
# dot product of any two of them should be 0.
# Let's check!

dot_prod01 = 0
for i in range(6):
    dot_prod01+=pca.components_[0][i] * pca.components_[1][i]
dot_prod01    

In [ ]:
dot_prod02 = 0
for i in range(6):
    dot_prod02+=pca.components_[0][i] * pca.components_[2][i]
dot_prod02

In [ ]:
dot_prod12 = 0
for i in range(6):
    dot_prod12+=pca.components_[1][i] * pca.components_[2][i]
dot_prod12 

## Visualizations

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

fig, ax = plt.subplots()
ax.plot(X_train_new[:, 0], y_train, 'r.');

In [ ]:
fig, ax = plt.subplots()
ax.plot(X_train_new[:, 1], y_train, 'g.');

In [ ]:
fig, ax = plt.subplots()
ax.plot(X_train_new[:, 2], y_train, 'k.');

Question: Is the first principal component the same line we would get if we constructed an ordinary least-squares regression line?

The answer is NO! Check out this post for an illuminating discussion: https://shankarmsy.github.io/posts/pca-vs-lr.html

## Modeling with New Dimensions

Now that we have optimized our features, we can build a new model with them!

In [ ]:
lr_pca = LinearRegression()
lr_pca.fit(X_train_new, y_train)
lr_pca.score(X_train_new, y_train)

In [ ]:
X_test_new = pca.transform(X_ts_sc)

In [ ]:
lr_pca.score(X_test_new, y_test)

In [ ]:
lr_pca.coef_    #-3.00 is the coefficient of the model but 0.454 was the coefficient of transformation

Thus, our best-fit hyperplane is given by:

$-3.00\times PC1 - 1.15\times PC2 -2.49\times PC3$

Of course, since the principal components are just linear combinations of our original predictors, we could re-express this hyperplane in terms of those original predictors!

And if the PCA was worth anything, we should expect the new linear model to be *different from* the first!

Recall that we had:

**PC1** = 0.454 * cylinders + 0.470 * cubicinches + 0.462 * hp + 0.440 * weightlbs - 0.357 * time-to-60 - 0.196 * year

**PC2** = -0.143 * cylinders - 0.110 * cubicinches - 0.023 * hp - 0.217 * weightlbs - 0.102 * time-to-60 - 0.954 * year

**PC3** = 0.204 * cylinders + 0.153 * cubicinches - 0.129 * hp + 0.361 * weightlbs + 0.860 * time-to-60 - 0.220 * year

Therefore, our new PCA-made hyperplane can be expressed as:

$-3.00\times(0.454\times cyl + 0.470\times in^3 + 0.462\times hp + 0.440\times lbs. - 0.357\times time_{60} - 0.196\times yr)$ <br/> $- 1.15\times(-0.143\times cyl - 0.110\times in^3 - 0.023\times hp - 0.217\times lbs. - 0.102\times time_{60} - 0.954\times yr)$ <br/> $- 2.49\times(0.204\times cyl + 0.153\times in^3 - 0.129\times hp + 0.361\times lbs. + 0.860\times time_{60} - 0.220\times yr)$ <br/><br/> $= -1.706\times cyl - 1.664\times in^3 -1.038\times hp - 1.969\times lbs. -3.095\times time_{60} + 2.233\times yr$

Our first linear regression model had:

$-1.404\times cyl + 0.671\times in^3 - 0.478\times hp - 4.655\times lbs. -  0.177\times time_{60} + 2.426\times yr$,
